In [ ]:
import scipy as sp, numpy as np
from sklearn.preprocessing import scale, StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import LeaveOneGroupOut, LeaveOneOut
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import copy

# Manipulate the Data into a Usable Format

For X matrix:

In [ ]:
X0 = sp.io.loadmat('X48_0T.mat')['X_data_0T']
X_avg = copy.deepcopy(X0)
num_col = len(X0[0, :])
num_row = len(X0[:, 0])
for col in range(num_col): # columns
    Num_NonZero = 0
    for row in range(num_row): # rows
        if X0[row, col] == 0:
            Num_NonZero = Num_NonZero + 1
    if Num_NonZero != 25:
        Row_Avg = np.mean(X0[:,col])*25/(25-Num_NonZero)

        for row in range(num_row): # rows
            if X0[row, col] == 0:
                X_avg[row, col] = Row_Avg;
print('X with zeros\n\n', X0)
print('\nX with averages\n\n', X_avg)

For Y matrix:

In [ ]:
Y = sp.io.loadmat('Y48_T.mat')['Y_data_T'] # function_workspace isn't right

Time48 = Y[:, 0] # measurement time
Dose48 = Y[:,1] # radiation exposure in Gray
Patients48 = Y[:, 2]; # patient number

# Get indices of each dosage
i0_5 = np.where(Dose48 == 0.5)[0]
i0 = np.where(Dose48 == 0)[0]
i2 = np.where(Dose48 == 2)[0]
i5 = np.where(Dose48 == 5)[0]
i8 = np.where(Dose48 == 8)[0]

Normalize both the X and Y data for PLSR

In [ ]:
X0norm = np.zeros((25,41094))
for c in range(41094):
    if np.sum(X0[:,c]) != 0:
        X0norm[:,c] = sp.stats.zscore(X0[:,c])  # normalize the X data
Dose48norm = sp.stats.zscore(Dose48)  # normalize the Y data

Do PLSR on the normalized data:

In [ ]:
variance_array = np.zeros(10) # calculate variance with different numbers of PLSR
for i in range(1,11):
    plsri = PLSRegression(n_components=i)
    plsri.fit(X0norm,Dose48norm)
    variance_array[i-1] = 100*plsri.score(X0norm,Dose48norm)
plt.scatter(range(1,11),variance_array)

# need to use plsri.predict in order to predict -> use arbitrary thresholds to categorize into 
#plt.plot(range(1,11),line)
plt.xlabel("Principal Components")
plt.ylabel("R^2")
plt.show()